In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from dataset.dataset_utils import *
from utils.runctx import RunCtx

In [4]:
run_ctx = RunCtx("<YOUR_TOKEN>")

In [5]:
all_topic_names = run_ctx.state.load("all_topic_names.json") 
results_v1_1 = run_ctx.state.load("categorization_results_v2.json")

In [11]:
from pipeline.pipeline import async_topic_mapping_pipeline

async def async_run_pipeline(model, prompt_fn, dataset, output_name):
    """
    This function runs a specified prompt and model with topic_mapping_pipeline against the batch of topics
    :param model: model to use 
    :param prompt_fn: prompt to use
    :param dataset: dataset to run on
    :param output_name: base name for results files
    """
    pipeline_fn = async_topic_mapping_pipeline(model, results_v1_1, prompt_fn)
    results = await run_ctx.run_async(pipeline_fn, dataset)
    run_ctx.state.save(results, output_name + ".json", overwrite=True)
    return results

In [12]:
from prompts.prompts import map_category_v1
topic_mapping_all = await async_run_pipeline(run_ctx.model_gpt3_5, map_category_v1, all_topic_names, "topic_mapping_all")

In [31]:
from dataset.dataset_utils import build_topic_sentiments_df

all_reviews_df = load_reviews(run_ctx.data_dir + "/us_reviews.csv")
topic_extraction_results = run_ctx.state.load("results_all_gpt_3_5_final.json")
 
topic_sentiments_df = build_topic_sentiments_df(
    all_reviews_df,
    topic_extraction_results, 
    topic_mapping_all
)
topic_sentiments_df.to_csv(run_ctx.data_dir + "/topic_sentiments.csv", index=False)